In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import getpass

username = getpass.getuser()

In [3]:
spark = SparkSession.\
        builder.\
        config('spark.ui.port','0').\
        config('spark.sql.warehouse.dir',f'/user/{username}/warehouse').\
        config('spark.shuffle.useOldFetchProtocol','true').\
        enableHiveSupport().\
        master('yarn').\
        getOrCreate()


In [4]:
raw_df = spark.read.csv("/public/trendytech/datasets/accepted_2007_to_2018Q4.csv" , header=True , inferSchema = True)

In [8]:
transformed_df = raw_df.withColumn("member_id" , sha2(concat_ws("||" , *["emp_title","emp_length","home_ownership","annual_inc","zip_code","addr_state","grade","sub_grade","verification_status"]) , 256))

In [14]:
transformed_df.createOrReplaceTempView("lending_club_data")

In [15]:
spark.sql("""
 select member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,'USA' as country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
 from lending_club_data   
""").repartition(1).write.format("csv").option("header",True).mode("overwrite").option("path" , "/user/itv015278/lendingclubproject/raw/customers_data_csv").save()

In [16]:
spark.sql("""
 select id as loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title
 from lending_club_data   
""").repartition(1).write.format("csv").option("header",True).mode("overwrite").option("path" , "/user/itv015278/lendingclubproject/raw/loans_data_csv").save()

In [17]:
spark.sql("""
 select id as loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
 from lending_club_data   
""").repartition(1).write.format("csv").option("header",True).mode("overwrite").option("path" , "/user/itv015278/lendingclubproject/raw/loans_repayments_csv").save()

In [19]:
spark.sql("""
 select member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
 from lending_club_data   
""").repartition(1).write.format("csv").option("header",True).mode("overwrite").option("path" , "/user/itv015278/lendingclubproject/raw/loans_defaulters_csv").save()